## Boilerplate e API

In [ ]:
import json
import io
import base64
import html
import numpy as np
import pandas as pd
import requests
from urllib.request import urlopen
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties

from highlight_text import fig_text
from mplsoccer import Bumpy, FontManager, add_image

## Criação do HTML

In [ ]:
def generate_html_clean(df_classificacao, df_jogos, b64_logo, img_str):
    html_content = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Tabela - Brasileirão Série B</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Roboto', sans-serif;
                margin: 0;
                padding: 0;
                background: #EFEFEF;
                padding-bottom: 60px;
            }}
            .header {{
                background-color: #C8102E;
                color: #F7B500;
                text-align: center;
                padding: 15px;
                position: relative;
                border-bottom: 3px solid #000000;
            }}
            .logo {{
                position: absolute;
                top: 10px;
                left: 10px;
                width: 90px;
                height: auto;
            }}
            h1 {{
                margin: 0;
                font-size: 3em;
                font-weight: 700;
                color: #F7B500; 
                text-shadow: 2px 2px 4px #000000; 
            }}
            h2 {{
                margin: 5px 0 0;
                font-size: 1.5em;
                color: #F7B500;
                text-shadow: 1px 1px 2px #000000;
            }}
            .container {{
                display: flex;
                flex-wrap: wrap;
                justify-content: space-around;
                padding: 15px;
            }}
            .table-container {{
                width: 48%;
                margin: 10px 0;
                background: #FFF;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                border-radius: 4px;
                overflow: hidden;
            }}
            .wide-container {{
                width: 100%;
                margin: 10px 0;
                background: #FFF;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                border-radius: 4px;
                overflow: hidden;
            }}
            .table-title {{
                background-color: #C8102E;
                color: #F7B500;
                padding: 12px;
                font-size: 1.5em; 
                text-align: center;
                border-bottom: 1px solid #000000;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 0;
            }}
            table, th, td {{
                border: 1px solid #DDDDDD;
            }}
            th, td {{
                padding: 10px;
                text-align: center;
            }}
            th {{
                background-color: #F7B500;
                color: #C8102E;
                font-weight: 700;
            }}
            tr:nth-child(even) {{
                background-color: #F9F9F9;
            }}
            tr:nth-child(odd) {{
                background-color: #EAEAEA;
            }}
            tr:hover {{
                background-color: #F1F1F1;
            }}
            .highlight-sport td {{
                background-color: #C8102E; 
                color: #F7B500; 
                font-weight: bold;
                text-shadow: 1px 1px 2px #000000;
            }}
            .footer {{
                text-align: center;
                padding: 8px;
                background-color: #C8102E;
                color: #F7B500;
                position: relative;
                bottom: 0;
                width: 100%;
                border-top: 1px solid #DDDDDD;
            }}
            .time-column {{
                text-align: left;
                padding-left: 20px;
                font-weight: bold;
            }}
            .wide-title {{
                background-color: #C8102E;
                color: #F7B500;
                padding: 12px;
                font-size: 1.5em;
                text-align: center;
                border-bottom: 1px solid #000000;
            }}
        </style>
    </head>
    <body>
        <div class="header">
            <img src="data:image/png;base64,{b64_logo}" alt="Logo da Leões de Sampa" class="logo"> 
            <h1>Tabela - Brasileirão Série B</h1>
            <h2>Leões de Sampa</h2>
        </div>
        <div class="container">
            <div class="table-container">
                <div class="table-title">Classificação Geral</div>
                <table>
                    <thead>
                        <tr>
                            <th>#</th>
                            <th class="time-column">Time</th>
                            <th>P</th>
                            <th>J</th>
                            <th>V</th>
                            <th>E</th>
                            <th>D</th>
                            <th>Gols</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    for index, row in df_classificacao.iterrows():
        row_class = 'highlight-sport' if row['Time'] == 'Sport' else ''
        time_name = html.escape(row['Time'])
        gol_stats = html.escape(row['Gols'])
        html_content += f"""
            <tr class="{row_class}">
                <td>{row['#']}</td>
                <td class="time-column">{time_name}</td>
                <td>{row['P']}</td>
                <td>{row['J']}</td>
                <td>{row['V']}</td>
                <td>{row['E']}</td>
                <td>{row['D']}</td>
                <td>{gol_stats}</td>
            </tr>
        """
    
    html_content += """
                    </tbody>
                </table>
            </div>
            <div class="table-container">
                <div class="table-title">Resultados da Rodada 21</div>
                <table>
                    <thead>
                        <tr>
                            <th>Mandante</th>
                            <th>Visitante</th>
                            <th>Placar</th>
                        </tr>
                    </thead>
                    <tbody>
    """
    
    for index, row in df_jogos.iterrows():
        row_class = 'highlight-sport' if row['Mandante'] == 'Sport' or row['Visitante'] == 'Sport' else ''
        mandante = html.escape(row['Mandante'])
        visitante = html.escape(row['Visitante'])
        placar = f"{row['Placar Mandante']} - {row['Placar Visitante']}"
        html_content += f"""
            <tr class="{row_class}">
                <td>{mandante}</td>
                <td>{visitante}</td>
                <td>{placar}</td>
            </tr>
        """
    
    html_content += f"""
                    </tbody>
                </table>
            </div>
            <div class="wide-container">
                <div class="wide-title">Gráfico de Desempenho do Sport</div>
                <div style="text-align: center;">
                <img src="data:image/png;base64,{img_str}" alt="Gráfico de Desempenho do Sport" style="max-width: 100%; height: auto; border: 2px solid #F7B500;">
                </div>
            </div>
            <div class="footer">
                <p>© 2024 Leões de Sampa. Todos os direitos reservados. Por Ana Gabriela S. Lima.</p>
            </div>
        </body>
        </html>
    """
    return html_content 


In [ ]:
html_content = generate_html_clean(df_estatisticas, df_ultima_rodada, b64_string, img_str)



In [ ]:
with open('tabela_sport_recife.html', 'w', encoding='utf-8') as file:
    file.write(html_content)

print("HTML gerado e salvo como 'tabela_sport_recife.html'")